## Problem 1.5

In [1]:
import datetime, time
import pytz
import json
import matplotlib.pyplot as plt
import io
import math
from collections import defaultdict
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import KFold

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

C:\Users\dell\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
testdata_before = ['test_data/sample1_period1.txt','test_data/sample4_period1.txt','test_data/sample5_period1.txt','test_data/sample8_period1.txt']
testdata_between = ['test_data/sample2_period2.txt','test_data/sample6_period2.txt','test_data/sample9_period2.txt']
testdata_after = ['test_data/sample3_period3.txt','test_data/sample7_period3.txt','test_data/sample10_period3.txt']

In [3]:
tag = ['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt', 
       'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']

In [4]:
starttime_list = np.zeros(6,int)
endtime_list = np.zeros(6,int)

In [5]:
for i in range(len(tag)):
    tagname = tag[i]

    print(tagname)
    f = io.open(tagname, 'rb')
    
    f.seek(0, 0)
    tweet_0 = json.loads(f.readline()) 
   
    f.seek(0, 0) #Go back to the original point
    
    starttime_list[i] = tweet_0['firstpost_date']
    
    for line in f:
        tweet = json.loads(line)
        end_time = tweet['firstpost_date']
    endtime_list[i] = end_time
print('start_time', starttime_list)
print('end_time', endtime_list)

tweets_#gohawks.txt
tweets_#gopatriots.txt
tweets_#nfl.txt
tweets_#patriots.txt
tweets_#sb49.txt
tweets_#superbowl.txt
start_time [1419804875 1420835445 1419999683 1419805279 1421238675 1419866833]
end_time [1423304269 1423295675 1423335336 1423335300 1423335336 1423332008]


In [6]:
start_time = min(starttime_list)
end_time = max(endtime_list)
time_8am = 1422806400
time_8pm = 1422849600
all_hour_before = math.ceil((time_8am - start_time)/3600.)
all_hour_between = math.ceil((time_8pm - time_8am)/3600.)
all_hour_after = math.ceil((end_time - time_8am)/3600.)

In [7]:
all_tweet_features_before = np.zeros([all_hour_before, 9])
all_user_id_before = [[]] * all_hour_before
all_tweet_features_between = np.zeros([all_hour_between, 9]) 
all_user_id_between = [[]] * all_hour_between
all_tweet_features_after = np.zeros([all_hour_after, 9])
all_user_id_after = [[]] * all_hour_after

In [8]:
# aggregate data features
for tagname in tag:
    f = io.open(tagname, 'r')
    
    # calculate features for model construction
    pst_tz = pytz.timezone('US/Pacific')

    for line in f:
        tweet = json.loads(line)
        time = tweet['firstpost_date']
        
        # For the first time period
        if time < 1422806400:
            # Calculate which hour current tweet belongs to
            n = math.ceil((time - start_time)/3600.)
            # Update value of features for corresponding hour 
            all_tweet_features_before[(n-1), 0] += 1
            all_tweet_features_before[(n-1), 1] += tweet['metrics']['citations']['total']
            followers = tweet['author']['followers']
            all_tweet_features_before[(n-1), 2] += followers
            if followers > all_tweet_features_before[(n-1), 3]:
                all_tweet_features_before[(n-1), 3] = followers
            time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
            all_tweet_features_before[(n-1), 4] = time_of_the_day
            all_tweet_features_before[(n-1), 5] += tweet['metrics']['ranking_score']
            all_tweet_features_before[(n-1), 6] += tweet['tweet']['favorite_count']
            all_tweet_features_before[(n-1), 7] += tweet['metrics']['impressions']
            all_user_id_before[(n-1)].append(tweet['tweet']['user']['id'])
            
        elif (time >= 1422806400) and (time < 1422849601):
            # Calculate which hour current tweet belongs to
            n = math.ceil((time - time_8am)/3600.)
            # Update value of features for corresponding hour 
            all_tweet_features_between[(n-1), 0] += 1
            all_tweet_features_between[(n-1), 1] += tweet['metrics']['citations']['total']
            followers = tweet['author']['followers']
            all_tweet_features_between[(n-1), 2] += followers
            if followers > all_tweet_features_between[(n-1), 3]:
                all_tweet_features_between[(n-1), 3] = followers
            time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
            all_tweet_features_between[(n-1), 4] = time_of_the_day
            all_tweet_features_between[(n-1), 5] += tweet['metrics']['ranking_score']
            all_tweet_features_between[(n-1), 6] += tweet['tweet']['favorite_count']
            all_tweet_features_between[(n-1), 7] += tweet['metrics']['impressions']
            all_user_id_between[(n-1)].append(tweet['tweet']['user']['id'])
            
        else:
            # Calculate which hour current tweet belongs to
            n = math.ceil((time - time_8pm)/3600.)
            # Update value of features for corresponding hour 
            all_tweet_features_after[(n-1), 0] += 1
            all_tweet_features_after[(n-1), 1] += tweet['metrics']['citations']['total']
            followers = tweet['author']['followers']
            all_tweet_features_after[(n-1), 2] += followers
            if followers > all_tweet_features_before[(n-1), 3]:
                all_tweet_features_after[(n-1), 3] = followers
            time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
            all_tweet_features_after[(n-1), 4] = time_of_the_day
            all_tweet_features_after[(n-1), 5] += tweet['metrics']['ranking_score']
            all_tweet_features_after[(n-1), 6] += tweet['tweet']['favorite_count']
            all_tweet_features_after[(n-1), 7] += tweet['metrics']['impressions']
            all_user_id_after[(n-1)].append(tweet['tweet']['user']['id'])
    
for t in range(len(all_user_id_before)):
    all_tweet_features_before[t, 8] = len(np.unique(all_user_id_before[t]))
for l in range(len(all_user_id_between)):
    all_tweet_features_between[l, 8] = len(np.unique(all_user_id_between[l]))
for n in range(len(all_user_id_after)):
    all_tweet_features_after[n, 8] = len(np.unique(all_user_id_after[n]))
print(sum(all_tweet_features_before[:, 0]))
print(sum(all_tweet_features_between[:, 0]))
print(sum(all_tweet_features_after[:, 0]))
    
    # Reconstruct independent and dependent variable
all_tweet_before_y = all_tweet_features_before[1:,0]
    #print(tweet_before_y.shape)
all_tweet_before_x = all_tweet_features_before[:(all_hour_before-1), 1:]
    #print(tweet_before_x.shape)
    
all_tweet_between_y = all_tweet_features_between[1:,0]
    #print(tweet_between_y.shape)
all_tweet_between_x = all_tweet_features_between[:(all_hour_between-1), 1:]
    #print(tweet_between_x.shape)
    
all_tweet_after_y = all_tweet_features_after[1:,0]
    #print(tweet_after_y.shape)
all_tweet_after_x = all_tweet_features_after[:(all_hour_after-1), 1:]
    #print(tweet_after_x.shape)
                
time_series = ['Before Feb. 1, 8:00 a.m.', 'Between Feb. 1, 8:00 a.m. and 8:00 p.m.', 'After Feb. 1, 8:00 p.m.']
tweet_y_all = [all_tweet_before_y, all_tweet_between_y, all_tweet_after_y]
tweet_x_all = [all_tweet_before_x, all_tweet_between_x, all_tweet_after_x]

671942.0
2193017.0
273864.0


#### trainset prepare

In [97]:
#before period
tweet_features_before_set_x = []
tweet_featuers_before_set_y = all_tweet_features_before[5:,0]

for i in range(all_hour_before-5):
    tweet_features_before_set_x.append(all_tweet_features_before[i:i+5,1:])
tweet_features_before_set_x=np.reshape(tweet_features_before_set_x,(len(tweet_features_before_set_x),40))

In [95]:
#between period
tweet_features_between_set_x = []
tweet_featuers_between_set_y = all_tweet_features_between[5:,0]

for i in range(all_hour_between-5):
    tweet_features_between_set_x.append(all_tweet_features_between[i:i+5,1:])
tweet_features_between_set_x=np.reshape(tweet_features_between_set_x,(len(tweet_features_between_set_x),40))

In [96]:
#after period
tweet_features_after_set_x = []
tweet_featuers_after_set_y = all_tweet_features_after[5:,0]

print(all_hour_after)
for i in range(all_hour_after-5):
    tweet_features_after_set_x.append(all_tweet_features_after[i:i+5,1:])
tweet_features_after_set_x=np.reshape(tweet_features_after_set_x,(len(tweet_features_after_set_x),40))

147


In [65]:
#gather information of testset
start_time_test_before = np.zeros(4,int)
end_time_test_before = np.zeros(4,int)
for i in range(len(testdata_before)):
    tagname = testdata_before[i]

    print(tagname)
    f = io.open(tagname, 'rb')
    
    f.seek(0, 0)
    tweet_0 = json.loads(f.readline()) 
   
    f.seek(0, 0) #Go back to the original point
    
    start_time_list_test[i] = tweet_0['firstpost_date']
    
    for line in f:
        tweet = json.loads(line)
        end_time = tweet['firstpost_date']
    end_time_list_test[i] = end_time
print('start_time', start_time_list_test)
print('end_time', end_time_list_test)

test_data/sample1_period1.txt
test_data/sample4_period1.txt
test_data/sample5_period1.txt
test_data/sample8_period1.txt
start_time [1422554405 1422223204 1422406820 1422489605]
end_time [1422575945 1422244781 1422428389 1422507351]


In [55]:
start_time = min(start_time_list_test)
end_time = max(end_time_list_test)
time_8am = 1422806400
time_8pm = 1422849600
test_hour_before = math.ceil((time_8am - start_time)/3600.)
test_hour_between = math.ceil((time_8pm - time_8am)/3600.)
test_hour_after = math.ceil((end_time - time_8am)/3600.)

In [114]:
RMSE_testing = []
all_test_before = len(testdata_before)
valid_test = [0,1,2]
for i in valid_test:
    sample = testdata_before[i]
    f = io.open(sample, 'r')
    print(i)
    start_time = start_time_list_test[i]
    time_end = end_time_list_test[i]
    test_hour_before = math.ceil((time_end - start_time)/3600.)
    print(test_hour_before)
    test_tweet_features_before = np.zeros([test_hour_before, 9])
    test_user_id_before = [[]] * test_hour_before
    
    # calculate features for model construction
    pst_tz = pytz.timezone('US/Pacific')
    tweet_0 = json.loads(f.readline())
    start_time_1 = tweet_0['firstpost_date']
    f.seek(0,0)
    for line in f:
        tweet = json.loads(line)
        time_test = tweet['firstpost_date']
        #print(time_test)
        #print(start_time_1)
        # Calculate which hour current tweet belongs to
        n = math.ceil((time_test - start_time_1)/3600.)
            # Update value of features for corresponding hour 
        test_tweet_features_before[(n-1), 0] += 1
        test_tweet_features_before[(n-1), 1] += tweet['metrics']['citations']['total']
        followers = tweet['author']['followers']
        test_tweet_features_before[(n-1), 2] += followers
        if followers >test_tweet_features_before[(n-1), 3]:
            test_tweet_features_before[(n-1), 3] = followers
        time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
        test_tweet_features_before[(n-1), 4] = time_of_the_day
        test_tweet_features_before[(n-1), 5] += tweet['metrics']['ranking_score']
        test_tweet_features_before[(n-1), 6] += tweet['tweet']['favorite_count']
        test_tweet_features_before[(n-1), 7] += tweet['metrics']['impressions']
        test_user_id_before[(n-1)].append(tweet['tweet']['user']['id'])
    f.close()
    #make testset
    for t in range(len(test_user_id_before)):
        test_tweet_features_before[t, 8] = len(np.unique(all_user_id_before[t]))
       
    tweet_features_before_test_x = []
    tweet_featuers_before_test_y = test_tweet_features_before[test_hour_before-1:,0]
    for i in range(1):
        tweet_features_before_test_x.append(test_tweet_features_before[i:i+test_hour_before-1,1:])
    tweet_features_before_test_x = np.reshape(tweet_features_before_test_x,(len(tweet_features_before_test_x),8*(test_hour_before-1)))
    # random forest built and fit to training set
    rf = RandomForestRegressor(criterion='mse')
    X_train = list(tweet_features_before_set_x)
    y_train = list(tweet_featuers_before_set_y)
    rf.fit(X_train, y_train)
    test_prediction = rf.predict(tweet_features_before_test_x)
    RMSE_testing.append(math.sqrt(metrics.mean_squared_error(test_prediction,tweet_featuers_before_test_y)))

    print(RMSE_testing[i],test_prediction,tweet_featuers_before_test_y)

0
6
2.5 [180.5] [178.]
1
6
2.5 [294.8] [203.]
2
6
2.5 [271.4] [211.]


In [115]:
#between
RMSE_testing_between = []
all_test_before = len(testdata_before)
valid_test = [0,1,2]
for i in valid_test:
    sample = testdata_between[i]
    f = io.open(sample, 'r')
    print(i)
    start_time = start_time_list_test[i]
    time_end = end_time_list_test[i]
    test_hour_between = math.ceil((time_end - start_time)/3600.)
    print(test_hour_between)
    test_tweet_features_between = np.zeros([test_hour_between, 9])
    test_user_id_between = [[]] * test_hour_between
    
    # calculate features for model construction
    pst_tz = pytz.timezone('US/Pacific')
    tweet_0 = json.loads(f.readline())
    start_time_1 = tweet_0['firstpost_date']
    f.seek(0,0)
    for line in f:
        tweet = json.loads(line)
        time_test = tweet['firstpost_date']
        #print(time_test)
        #print(start_time_1)
        # Calculate which hour current tweet belongs to
        n = math.ceil((time_test - start_time_1)/3600.)
            # Update value of features for corresponding hour 
        test_tweet_features_between[(n-1), 0] += 1
        test_tweet_features_between[(n-1), 1] += tweet['metrics']['citations']['total']
        followers = tweet['author']['followers']
        test_tweet_features_between[(n-1), 2] += followers
        if followers >test_tweet_features_between[(n-1), 3]:
            test_tweet_features_between[(n-1), 3] = followers
        time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
        test_tweet_features_between[(n-1), 4] = time_of_the_day
        test_tweet_features_between[(n-1), 5] += tweet['metrics']['ranking_score']
        test_tweet_features_between[(n-1), 6] += tweet['tweet']['favorite_count']
        test_tweet_features_between[(n-1), 7] += tweet['metrics']['impressions']
        test_user_id_between[(n-1)].append(tweet['tweet']['user']['id'])
    f.close()
    #make testset
    for t in range(len(test_user_id_between)):
        test_tweet_features_between[t, 8] = len(np.unique(all_user_id_between[t]))
       
    tweet_features_between_test_x = []
    tweet_featuers_between_test_y = test_tweet_features_between[test_hour_between-1:,0]
    for i in range(1):
        tweet_features_between_test_x.append(test_tweet_features_between[i:i+test_hour_between-1,1:])
    tweet_features_between_test_x = np.reshape(tweet_features_between_test_x,(len(tweet_features_between_test_x),8*(test_hour_between-1)))
    
    rf = RandomForestRegressor()
    X_train = list(tweet_features_between_set_x)
    y_train = list(tweet_featuers_between_set_y)
    rf.fit(X_train, y_train)
    test_prediction = rf.predict(tweet_features_between_test_x)
    RMSE_testing_between.append(math.sqrt(metrics.mean_squared_error(test_prediction,tweet_featuers_between_test_y)))
    print(RMSE_testing_between[i],test_prediction,tweet_featuers_between_test_y)
    

0
6
211097.5 [293989.5] [82892.]
1
6
211097.5 [310178.2] [37279.]
2
6
211097.5 [249774.5] [2791.]


In [136]:
#after
RMSE_testing_after = []
all_test_before = len(testdata_before)
valid_test = [0,1,2]
for i in valid_test:
    sample = testdata_after[i]
    f = io.open(sample, 'r')
    print(i)
    start_time = start_time_list_test[i]
    time_end = end_time_list_test[i]
    test_hour_after = math.ceil((time_end - start_time)/3600.)
    print(test_hour_after)
    test_tweet_features_after = np.zeros([test_hour_after, 9])
    test_user_id_after = [[]] * test_hour_after
    
    # calculate features for model construction
    pst_tz = pytz.timezone('US/Pacific')
    tweet_0 = json.loads(f.readline())
    start_time_1 = tweet_0['firstpost_date']
    f.seek(0,0)
    for line in f:
        tweet = json.loads(line)
        time_test = tweet['firstpost_date']
        #print(time_test)
        #print(start_time_1)
        # Calculate which hour current tweet belongs to
        n = math.ceil((time_test - start_time_1)/3600.)
            # Update value of features for corresponding hour 
        test_tweet_features_after[(n-1), 0] += 1
        test_tweet_features_after[(n-1), 1] += tweet['metrics']['citations']['total']
        followers = tweet['author']['followers']
        test_tweet_features_after[(n-1), 2] += followers
        if followers >test_tweet_features_after[(n-1), 3]:
            test_tweet_features_after[(n-1), 3] = followers
        time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
        test_tweet_features_after[(n-1), 4] = time_of_the_day
        test_tweet_features_after[(n-1), 5] += tweet['metrics']['ranking_score']
        test_tweet_features_after[(n-1), 6] += tweet['tweet']['favorite_count']
        test_tweet_features_after[(n-1), 7] += tweet['metrics']['impressions']
        test_user_id_after[(n-1)].append(tweet['tweet']['user']['id'])
    f.close()
    #make testset
    for t in range(len(test_user_id_after)):
        test_tweet_features_after[t, 8] = len(np.unique(all_user_id_after[t]))
       
    tweet_features_after_test_x = []
    tweet_featuers_after_test_y = test_tweet_features_after[test_hour_after-1:,0]
    for i in range(1):
        tweet_features_after_test_x.append(test_tweet_features_after[i:i+test_hour_after-1,1:])
    tweet_features_after_test_x = np.reshape(tweet_features_after_test_x,(len(tweet_features_after_test_x),8*(test_hour_after-1)))
    
    rf = RandomForestRegressor(n_estimators=20)
    X_train = list(tweet_features_after_set_x)
    y_train = list(tweet_featuers_after_set_y)
    rf.fit(X_train, y_train)
    test_prediction = rf.predict(tweet_features_after_test_x)
    RMSE_testing_after.append(math.sqrt(metrics.mean_squared_error(test_prediction,tweet_featuers_after_test_y)))
    print(RMSE_testing_after[i],test_prediction,tweet_featuers_after_test_y)


0
6
460.4 [984.4] [524.]
1
6
460.4 [31.95] [121.]
2
6
460.4 [49.1] [62.]


In [138]:
#special model for sample8_period1.txt
tweet_features_before_set_x_ = []
tweet_featuers_before_set_y_ = all_tweet_features_before[4:,0]

for i in range(all_hour_before-4):
    tweet_features_before_set_x_.append(all_tweet_features_before[i:i+4,1:])
tweet_features_before_set_x_=np.reshape(tweet_features_before_set_x_,(len(tweet_features_before_set_x_),32))

In [200]:
RMSE_testing = []
all_test_before = len(testdata_before)
valid_test = [3]
for i in valid_test:
    sample = testdata_before[i]
    f = io.open(sample, 'r')
    print(i)
    start_time = start_time_list_test[i]
    time_end = end_time_list_test[i]
    test_hour_before = math.ceil((time_end - start_time)/3600.)
    print(test_hour_before)
    test_tweet_features_before = np.zeros([test_hour_before, 9])
    test_user_id_before = [[]] * test_hour_before
    
    # calculate features for model construction
    pst_tz = pytz.timezone('US/Pacific')
    tweet_0 = json.loads(f.readline())
    start_time_1 = tweet_0['firstpost_date']
    f.seek(0,0)
    for line in f:
        tweet = json.loads(line)
        time_test = tweet['firstpost_date']
        #print(time_test)
        #print(start_time_1)
        # Calculate which hour current tweet belongs to
        n = math.ceil((time_test - start_time_1)/3600.)
            # Update value of features for corresponding hour 
        test_tweet_features_before[(n-1), 0] += 1
        test_tweet_features_before[(n-1), 1] += tweet['metrics']['citations']['total']
        followers = tweet['author']['followers']
        test_tweet_features_before[(n-1), 2] += followers
        if followers >test_tweet_features_before[(n-1), 3]:
            test_tweet_features_before[(n-1), 3] = followers
        time_of_the_day = int(datetime.datetime.fromtimestamp(tweet['firstpost_date'], pst_tz).strftime("%H"))
        test_tweet_features_before[(n-1), 4] = time_of_the_day
        test_tweet_features_before[(n-1), 5] += tweet['metrics']['ranking_score']
        test_tweet_features_before[(n-1), 6] += tweet['tweet']['favorite_count']
        test_tweet_features_before[(n-1), 7] += tweet['metrics']['impressions']
        test_user_id_before[(n-1)].append(tweet['tweet']['user']['id'])
    f.close()
    #make testset
    for t in range(len(test_user_id_before)):
        test_tweet_features_before[t, 8] = len(np.unique(all_user_id_before[t]))
       
    tweet_features_before_test_x = []
    tweet_featuers_before_test_y = test_tweet_features_before[test_hour_before-1:,0]
    for i in range(1):
        tweet_features_before_test_x.append(test_tweet_features_before[i:i+test_hour_before-1,1:])
    tweet_features_before_test_x = np.reshape(tweet_features_before_test_x,(len(tweet_features_before_test_x),8*(test_hour_before-1)))
    # random forest built and fit to training set
    rf = RandomForestRegressor(n_estimators=10,criterion='mse')
    X_train = list(tweet_features_before_set_x_)
    y_train = list(tweet_featuers_before_set_y_)
    rf.fit(X_train, y_train)
    test_prediction = rf.predict(tweet_features_before_test_x)
    RMSE_testing.append(math.sqrt(metrics.mean_squared_error(test_prediction,tweet_featuers_before_test_y)))

    print(RMSE_testing[i],test_prediction,tweet_featuers_before_test_y)

3
5
37.6 [49.6] [12.]
